<a href="https://colab.research.google.com/github/paucimi/frigo-vision-recommender/blob/main/recuperar_y_taguear_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de la herramienta de descarga
!pip install fiftyone


In [ ]:
#esta parte hay que cambiarla para que lo tome de los parámetros de entrada de la web
# montar el entorno de drive - esto es para que me encuentren los ficheros,

from google.colab import drive
drive.mount('/content/drive')

# incorporar el nombre del proyecto
ruta_proyecto = '/content/drive/MyDrive/'
os.chdir(ruta_proyecto)
base_dir = "./fridge"


In [ ]:
# si queremos usar un bucket en lugar de drive,

#Primero, debes dar permiso a Colab para acceder a tu proyecto de Google Cloud:
#Ve a IAM & Admin > Service Accounts.
#Crea una cuenta (ej. "colab-service-account") y dale el rol de Editor o Vertex AI User y Storage Admin.
#En la pestaña Keys, haz clic en Add Key > Create new key y selecciona el formato JSON. Se descargará un archivo a tu ordenador.
#Sube ese archivo .json a la carpeta de archivos de Cola

import os

# 1. Ruta al archivo que acabas de subir (cambiarlo)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./miccColab.json"

# 2. Ahora ya puedes usar comandos !gcloud o !gsutil directamente
# (Nota: a veces gcloud necesita un login extra, pero las librerías de Python ya funcionarán)
project_id = "tu-proyecto-id"
!gcloud auth activate-service-account --key-file=$GOOGLE_APPLICATION_CREDENTIALS
!gcloud config set project {project_id}


#Configurar el ID de tu Proyecto
project_id = "TU_PROJECT_ID"
!gcloud config set project {project_id}

# Comandos para habilitar las APIs
# Habilitar Vertex AI API
!gcloud services enable aiplatform.googleapis.com

# Habilitar Cloud Storage API
!gcloud services enable storage.googleapis.com

# Generar el bucket
GOOGLE_CLOUD_PROJECT = project_id
base_dir = f"gs://{project_id}-fridge/"
!gsutil mb -p $GOOGLE_CLOUD_PROJECT -c standard -l us-central1 base_dir


In [ ]:

# Descargamos imágenes y etiquetas automáticamente
import fiftyone as fo
import fiftyone.zoo as foz
import os
import csv
import shutil

# 1. Definición de la lista y rutas
lista_alimentos = [
    "Sauce",           # Mayonesa y salsas en frascos
    "Container",       # Para platos preparados, tupperwares y recipientes desconocidos
# --- Lácteos y Bebidas ---
    "Juice",            # Zumos
    "Beer",             # Cervezas
    "Wine",             # Vino
    "Cream",            # Nata o cremas
    "Butter",           # Mantequilla
    "Cheese",          # Kéfir (Se asocia a lácteos similares)
     "Milk",
    "Yogurt",          # Yogures (Griegos, Bífidus, Naturales)

    # --- Verduras y Hortalizas (Vegetables) ---
    "Tomato",           # Tomates
    "Cucumber",         # Pepino
    "Broccoli",         # Brócoli
    "Zucchini",         # Calabacín
    "Bell pepper",      # Pimiento
    "Potato",           # Patatas (a veces se guardan fuera, pero es útil)
    "Cabbage",          # Repollo/Col
    "Lettuce",          # Lechuga
    "Carrot",          # Zanahorias
     "Onion",
    "Olive",           # Aceitunas

    # --- Frutas ---
    "Strawberry",       # Fresas
    "Grape",            # Uvas
    "Banana",           # Plátanos
    "Lemon",            # Limones
    "Pear",             # Peras
    "Peach",            # Melocotones
     "Orange",
     "Apple"

    # --- Proteínas y Carnes ---
    "Chicken",          # Pollo
    "Seafood",          # Mariscos/Pescado (etiqueta general)
    "Ham",              # Jamón / Embutidos
    "Beef",             # Carne de vacuno
    "Turkey",          # Pavo en lonchas (Aunque suele ser difícil, es la etiqueta oficial)
    "Fast food"        # Hamburguesas
    "Egg",             # Huevos
    "Hamburger"        # Hamburguesas

    # --- Otros ---
    "Bread",            # Pan (si se guarda en nevera)
    "Mushroom",         # Champiñones / Setas
    "Salad",            # Ensaladas preparadas
    "Jellyfish"         # (Curiosidad: Open Images la usa a veces para gelatinas)
]
csv_path = os.path.join(base_dir, "data.csv")

# Asegurar que la carpeta base existe
os.makedirs(base_dir, exist_ok=True)


# Preparar el archivo CSV (escribimos la cabecera)
with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["ruta_imagen", "alimento"])

print(f"Iniciando descarga y tagueado en {base_dir}...")

# 2. Bucle recursivo/iterativo por cada alimento
for alimento in lista_alimentos:
    print(f"\n--- Procesando: {alimento} ---")

    # Crear carpeta específica para el alimento
    alimento_dir = os.path.join(base_dir, alimento)
    os.makedirs(alimento_dir, exist_ok=True)

    try:
        # Descarga del dataset para el alimento específico
        dataset = foz.load_zoo_dataset(
            "open-images-v7",
            split="validation",
            classes=[alimento],
            max_samples=10, # Ajusta según necesites
            seed=51,
            drop_existing_dataset=True # Limpia memoria de descargas previas
        )

        # Identificar campo de etiquetas (detections o ground_truth)
        label_field = "detections" if "detections" in dataset.get_field_schema() else "ground_truth"

        # Exportar imágenes a la subcarpeta del alimento
        # Usamos el tipo ImageDirectory porque solo queremos las fotos organizadas
        dataset.export(
            export_dir=alimento_dir,
            dataset_type=fo.types.ImageDirectory,
            label_field=label_field,
        )

        # 3. Generar líneas en el CSV para las imágenes descargadas
        with open(csv_path, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for filename in os.listdir(alimento_dir):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    ruta_completa = os.path.abspath(os.path.join(alimento_dir, filename))
                    writer.writerow([ruta_completa, alimento])

        print(f"✅ {alimento} completado y registrado en CSV.")

    except Exception as e:
        print(f"❌ Error procesando {alimento}: {e}")

print(f"\n🚀 Proceso finalizado. Archivo creado en: {csv_path}")